<a href="https://colab.research.google.com/github/Arianr0d/MORD/blob/%D0%98%D0%94%D0%97-1/%D0%98%D0%94%D0%971_%D0%9C%D0%9E%D0%A0%D0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import copy

## Реализация алгоритма

In [ ]:
# функция формирования матрицы со входными дугами
def transitionMatrix(input, output):
  return np.subtract(output, input)

# функция вывода результатов расчётов на каждом этапе
def printResultTransitions(node, sirealNode, keyNode):
  print('M{}({}) по переходу t{}'.format(keyNode, node, sirealNode + 1))

In [ ]:
def createSetNodes(startNode):
  return {
    '0': {
        'value': startNode,
        'parent': '',
    }
  }

setNodes = createSetNodes(np.array([]))

# функция сравнения векторов
def isGreaterVector(a, b):
  return np.greater_equal(a, b).all()

# функция проверки тупика
def isDeadlockTransition(serialTransition):
  notExistsInput = len([item for item in matrixRInput[serialTransition] if item == 0]) == countPosition
  return notExistsInput

# функция формирования узла с бесконечностью
def formationInfinityNode(a, b):
  nodeWithInfinity = []
  relationBooleanList = np.greater(a, b)

  for index in range(len(relationBooleanList)):
    if relationBooleanList[index]:
      nodeWithInfinity.append('w')
    else:
      nodeWithInfinity.append(a[index])

  return nodeWithInfinity

# функция проверки наличия совпадений в дереве разрешимости
def checkingRepetitionNodes(findationNode):
  nodeList = np.array([node['value'] for node in setNodes.values()])
  return any(np.equal(nodeList, findationNode).all(1))

# функция проверки на наличие бесконечности
def findInfinity(nodeKey):
  currentNode = setNodes[nodeKey]['value']
  previousNode = currentNode
  previousParent = setNodes[nodeKey]['parent']
  infinityNode = []

  while(True):
    if not setNodes[previousParent]['parent']:
      break

    previousNode = setNodes[previousParent]['value']
    previousParent = setNodes[previousParent]['parent']

    if isGreaterVector(currentNode, previousNode):
      infinityNode = formationInfinityNode(currentNode, previousNode)

  return infinityNode

# функция формирования ключа узла
def formationKey(depth, parent, serialTransition):
  parentTransitionList = parent.split('.')

  if len(parentTransitionList) > 0 and depth == 1:
    return '{}.{}'.format(depth, serialTransition + 1)
  elif depth > 1:
    intermediate = '.'.join(parentTransitionList[1:])
    return '{}.{}.{}'.format(depth, intermediate, serialTransition + 1)

# функция добавления узла в словарь
def addNodeInSet(key, node, parent):
  setNodes[key] = {
      'value': node,
      'parent': parent,
  }

In [ ]:
# поэтапный расчёт переходов
def nextTransitions(serialTransition, matrixRInput, previousNode, depth, parent, matrixR, E):
  mu = np.dot(E[serialTransition], matrixRInput)
  keyCurrentNode = formationKey(depth, parent, serialTransition)

  if not (isGreaterVector(previousNode, mu)) or isDeadlockTransition(serialTransition):
    print('M{} - переход t={} не разрешён!'.format(keyCurrentNode, serialTransition + 1))
    return

  currentNode = np.add(
      np.dot(E[serialTransition], matrixR),
      previousNode,
  )

  if checkingRepetitionNodes(currentNode):
    print('M{} - повторение'.format(keyCurrentNode))
    return
  else:
    addNodeInSet(keyCurrentNode, currentNode, parent)

  infinityNode = findInfinity(keyCurrentNode)
  if infinityNode:
    print('M{}({}) - переход t={} содержит бесконечность)'.format(keyCurrentNode, infinityNode, serialTransition))
    addNodeInSet(keyCurrentNode + 'with-w', currentNode, parent)
    return

  printResultTransitions(currentNode, serialTransition, keyCurrentNode)

  for transition in range(countTransition):
    nextTransitions(transition, matrixRInput, currentNode, depth + 1, keyCurrentNode, matrixR, E)

##Проверка свойств

In [ ]:
# функция поиска псевдообратной матрицы методом Фаддеева
def pseudoinverseFaddeev(matrix):
  countRowMatrix = matrix.shape[0]
  countColMatrix = matrix.shape[1]

  I = np.eye(countColMatrix)
  F = I
  phi = np.trace(np.dot(matrix.T, matrix))
  counter = 1

  while(phi != 0):
    counter+=1

    F_indeterminate = phi * I - np.dot(np.dot(matrix.T, matrix), F)
    phi_indeterminate = (np.trace(np.dot(np.dot(matrix.T, matrix), F_indeterminate)) / counter).astype(int)

    if phi_indeterminate == 0:
      F = np.dot(F, matrix.T)
      break
    else:
      F = F_indeterminate
      phi = phi_indeterminate

  if phi != 0:
    return F / phi
  else:
    print('Псевдообратная матрица не существует!')

In [ ]:
def generateRandomIndexes(countPosition):
  setNodesSize = len(setNodes)

  return np.array(random.sample(range(setNodesSize), countPosition + 1))

# функция проверки наличия бесконечности
def isInfinityNet():
  markerInfinity = 'with-w'

  for key in setNodes.keys():
    if markerInfinity in key:
      return True
  return False

# функция поиска максимального значения маркеровок
def findMaximumValueMarker():
  maximum = -np.Infinity

  for node in setNodes.values():
    indeterminateMax = max(node['value'])

    if indeterminateMax > maximum:
      maximum = indeterminateMax

  return maximum

# функция сравнения суммы маркеров в каждом переходе
def areEqualsSummaryAllMarkers():
  if len(setNodes) == 0:
    print('Список переходов пуст!')
    return

  summary = sum(setNodes['0']['value'])

  for node in setNodes.values():
    if summary != sum(node['value']):
      return False

  print('3. Сумма маркеров в каждом такте времени = {}'.format(summary))
  return True

# функция построения матрицы коэффициентов
def matrixOdds(randomIndexes):
  return np.array([setNodes[list(setNodes.keys())[index]]['value'] for index in randomIndexes])

# функция построения матрицы коэффициентов
def vectorB(randomIndex):
  return np.array([setNodes[list(setNodes.keys())[randomIndex]]['value'] for index in range(countPosition)])

def indexZerosRows(matrix):
  return np.where(~(matrix).any(axis=1))[0]

def indexZerosColumns(matrix):
  return np.where(~(matrix).any(axis=0))[0]

def indexesNoZerosColumn(matrix):
  return np.nonzero(np.any(matrix != 0, axis=0))[0]

# функция удаления нулевых строк и столбов
def deleteZeroLine(matrix):
  return np.delete(
    np.delete(matrix, indexZerosRows(matrix), axis=0),
    indexZerosColumns(matrix), axis=1
  )

# функция восстановления полного решения за счёт добавления случайного варианта из бесконечного множества решений
def restorationCompleteSolution(matrix, vector):
  indexPass = indexZerosColumns(matrix)

  if len(indexPass) > 0:
    fullVector = vector[:indexPass[0]]

    for i in range(len(indexPass)):
      fullVector = np.append(fullVector, 0.)

      if len(indexPass) > i + 1 and indexPass[i+1] - indexPass[i] != 1:
        fullVector = np.append(fullVector, vector[indexPass[i] - i : indexPass[i+1] - 1 - i])
      elif len(indexPass) == i + 1:
        fullVector = np.append(fullVector, vector[indexPass[i] - i:])

  return fullVector

# функция поиска решения системы уравнений через псевдообращение
def solvingSystemEquations(A, b):
  newMatrix = A

  if np.linalg.det(A).astype(int) == 0:
    newMatrix = deleteZeroLine(A)
    print('4. Матрица имеет бесконечное число решений! Одно из возможных решений -')

  preudoMatrix = pseudoinverseFaddeev(newMatrix)
  result = np.dot(preudoMatrix, b)
  indexesNoZerosCol = indexesNoZerosColumn(result)

  if len(indexesNoZerosCol) > 0:
    result = result[:, indexesNoZerosCol[0]]

  if (newMatrix.shape != A.shape):
    result = restorationCompleteSolution(A, result)

  return result

In [ ]:
# функция проверки ограниченности СП
def checkingLimitation():
  if isInfinityNet():
    print('1. СП не является ограниченной, так как содержит w')
  else:
    print('1. СП ограничена, S={}'.format(findMaximumValueMarker()))

# функция проверки безопасности СП
def checkingSafeness():
  if isInfinityNet():
    print('2. СП не является безопасной, так как содержит w')
  else:
    s = findMaximumValueMarker()

    if s == 1:
      print('2. СП безопасна')
    else:
      print('2. СП не безопасна, так как S={} и S!=1'.format(s))

# функция проверки строгого сохранения
def checkingStrictConservation():
  if isInfinityNet():
    print('3. СП не является строго сохраняющей, так как содержит w')
  else:
    if areEqualsSummaryAllMarkers():
      print('3. СП является строго сохраняющей и равно ')
    else:
      print('3. СП не является строго сохраняющей')

# функция проверки строго сохранения сети петри
def checkPassedOnConservation(a):
  vectorCoef = a.astype(int)
  stackNodes = [node['value'].astype(int) for node in setNodes.values()]

  summaryLabelsInNode = np.dot(stackNodes[0], vectorCoef)

  for node in stackNodes:
    if np.dot(node, vectorCoef) != summaryLabelsInNode:
      return False

  return True

# функция проверки сохранения
def checkingConservation(countPosition):
  if isInfinityNet():
    print('4. СП не является сохраняющей, так как содержит w')
  else:
    randomIndexes = generateRandomIndexes(countPosition)

    answerVector = solvingSystemEquations(
        matrixOdds(randomIndexes[:countPosition]),
        vectorB(randomIndexes[countPosition]),
    )

    isPassOnConservation = checkPassedOnConservation(np.array(answerVector).T)

    print('СП является сохраняющей, a = {}'.format(answerVector))
    if isPassOnConservation:
      print('Проверка пройдена успешно!')
    else:
      print('Проверка не пройдена!')

# функция проверки свойств
def checkingProperties(countPosition):
  checkingLimitation()
  checkingSafeness()
  checkingStrictConservation()
  checkingConservation(countPosition)

##Расчёт матриц и проверка их свойств

In [ ]:
# начальная маркировка
M0 = np.array([3,2,2,0,3])
# матрица со входными дугами в переходах
matrixRInput = np.array([[1,1,0,0,0],[0,0,1,0,0],[0,0,0,0,1]])
# матрица с выходными дугами в переходах
matrixROutput = np.array([[0,0,1,0,0],[1,1,0,1,1],[1,0,0,0,0]])
# количество переходов
countTransition = 3
# количество позиций
countPosition = 5

In [ ]:
setNodes = createSetNodes(M0)

print('M0([3 2 2 0 3])')
for transition in range(countTransition):
  nextTransitions(transition, matrixRInput, M0, 1, '0', transitionMatrix(matrixRInput, matrixROutput), np.eye(countTransition).astype(int))

M0([3 2 2 0 3])
M1.1([2 1 3 0 3]) по переходу t1
M2.1.1([1 0 4 0 3]) по переходу t1
M3.1.1.1 - переход t=1 не разрешён!
M3.1.1.2([2, 1, 3, 'w', 'w']) - переход t=1 содержит бесконечность)
M3.1.1.3([2 0 4 0 2]) по переходу t3
M4.1.1.3.1 - переход t=1 не разрешён!
M4.1.1.3.2(['w', 1, 3, 'w', 3]) - переход t=1 содержит бесконечность)
M4.1.1.3.3([3 0 4 0 1]) по переходу t3
M5.1.1.3.3.1 - переход t=1 не разрешён!
M5.1.1.3.3.2([4 1 3 1 2]) по переходу t2
M6.1.1.3.3.2.1(['w', 0, 4, 'w', 2]) - переход t=0 содержит бесконечность)
M6.1.1.3.3.2.2([5 2 2 2 3]) по переходу t2
M7.1.1.3.3.2.2.1(['w', 1, 3, 'w', 3]) - переход t=0 содержит бесконечность)
M7.1.1.3.3.2.2.2([6 3 1 3 4]) по переходу t2
M8.1.1.3.3.2.2.2.1([5, 2, 2, 'w', 'w']) - переход t=0 содержит бесконечность)
M8.1.1.3.3.2.2.2.2([7 4 0 4 5]) по переходу t2
M9.1.1.3.3.2.2.2.2.1([6, 3, 1, 'w', 'w']) - переход t=0 содержит бесконечность)
M9.1.1.3.3.2.2.2.2.2 - переход t=2 не разрешён!
M9.1.1.3.3.2.2.2.2.3([8 4 0 4 4]) по переходу t3
M10.1.1

In [ ]:
checkingProperties(countPosition)

1. СП не является ограниченной, так как содержит w
2. СП не является безопасной, так как содержит w
3. СП не является строго сохраняющей, так как содержит w
4. СП не является сохраняющей, так как содержит w


In [ ]:
# начальная маркировка
M0 = np.array([2,2,2,0,0])
# матрица со входными дугами в переходах
matrixRInput = np.array([[1,1,0,0,0],[0,0,0,0,0],[0,0,1,0,0],[1,0,0,0,0]])
# матрица с выходными дугами в переходах
matrixROutput = np.array([[0,0,0,0,0],[0,0,1,0,1],[0,1,0,0,0],[0,0,0,1,0]])
# количество переходов
countTransition = 4
# количество позиций
countPosition = 5

In [ ]:
setNodes = createSetNodes(M0)

print('M0([2 2 2 0 0])')
for transition in range(countTransition):
  nextTransitions(transition, matrixRInput, M0, 1, '0', transitionMatrix(matrixRInput, matrixROutput), np.eye(countTransition).astype(int))

M0([2 2 2 0 0])
M1.1([1 1 2 0 0]) по переходу t1
M2.1.1([0 0 2 0 0]) по переходу t1
M3.1.1.1 - переход t=1 не разрешён!
M3.1.1.2 - переход t=2 не разрешён!
M3.1.1.3([0 1 1 0 0]) по переходу t3
M4.1.1.3.1 - переход t=1 не разрешён!
M4.1.1.3.2 - переход t=2 не разрешён!
M4.1.1.3.3([0 2 0 0 0]) по переходу t3
M5.1.1.3.3.1 - переход t=1 не разрешён!
M5.1.1.3.3.2 - переход t=2 не разрешён!
M5.1.1.3.3.3 - переход t=3 не разрешён!
M5.1.1.3.3.4 - переход t=4 не разрешён!
M4.1.1.3.4 - переход t=4 не разрешён!
M3.1.1.4 - переход t=4 не разрешён!
M2.1.2 - переход t=2 не разрешён!
M2.1.3([1 2 1 0 0]) по переходу t3
M3.1.3.1 - повторение
M3.1.3.2 - переход t=2 не разрешён!
M3.1.3.3([1 3 0 0 0]) по переходу t3
M4.1.3.3.1 - повторение
M4.1.3.3.2 - переход t=2 не разрешён!
M4.1.3.3.3 - переход t=3 не разрешён!
M4.1.3.3.4([0 3 0 1 0]) по переходу t4
M5.1.3.3.4.1 - переход t=1 не разрешён!
M5.1.3.3.4.2 - переход t=2 не разрешён!
M5.1.3.3.4.3 - переход t=3 не разрешён!
M5.1.3.3.4.4 - переход t=4 не разре

In [ ]:
checkingProperties(countPosition)

1. СП ограничена, S=4
2. СП не безопасна, так как S=4 и S!=1
3. СП не является строго сохраняющей
4. Матрица имеет бесконечное число решений! Одно из возможных решений -
СП является сохраняющей, a = [-0.5  0.5  0.5 -0.5  0. ]
Проверка пройдена успешно!
